# Selection of Reference Stations for Central Italy

Team: Codellera Andina  
Authors:  
- María Fernanda Molina  
- Nataly Sarmiento  
- Isabela Saud

This notebook selects **10–15 high-quality reference weather stations** in **Central Italy** for the Urban Heat Island Hackathon (Period 3 – Data & Features).  
These stations are used to compute the **ERA5–station error**.

In this version we also **manually ensure** that three visually identified coastal rural stations  
(**TARQUINIA, ITRI, MONTALTO_PESCIA**) are included as **rural_coastal** reference stations.

---

## **Inputs**

- ECAD diagnostics (RMSE, coverage, correlation, number of good summers)  
- Valid station geometries (GPKG)  
- Central Italy AOI (shapefile)  
- Imperviousness around stations → `urban_fraction` (QGIS zonal stats)  
- `NDVI_mean_JJA` from Sentinel-2 (QGIS buffers, used descriptively)  
- European coastline (for `distance_to_sea_km`)  
- ERA5-Land grid (NetCDF, used to assign ERA5 cell IDs)

---

## **Method**

1. **Filter stations**  
   - Keep only stations flagged as valid. Valid are the station that have high quality data of at least 2 summers between 2020 and 2023
   - Clip to the Central Italy AOI.

2. **Merge spatial features**  
   - Compute `urban_fraction = imd_mean / 100` (0–1).  
   - Normalize NDVI (`NDVI_mean`) with min–max scaling (0–1).  
   - Compute `distance_to_sea_km` from the coastline (in EPSG:3035).

3. **Define urban vs rural from imperviousness (quantiles)**  
   - Compute quantiles of `urban_fraction` over the AOI:  
     - bottom 25% → **rural threshold** (`URBAN_LOW_THRESHOLD`)  
     - top 25%   → **urban threshold** (`URBAN_HIGH_THRESHOLD`)

4. **Assign ERA5 gridcell ID**  
   - Match each station to the nearest ERA5-Land grid point.  
   - Store `era5_cell_id` and later enforce **one station per ERA5 cell**.

5. **Initial UHI classification (automatic)**  
   - Use `urban_fraction` + `distance_to_sea_km` (< 10 km = coastal).  
   - Classes:  
     - `urban_coastal`, `urban_interior`  
     - `rural_coastal`, `rural_interior`  
     - `unknown` for mid-impervious stations.  

6. **Manual override: TARQUINIA, ITRI, MONTALTO_PESCIA**  
   - Print their diagnostics (quality metrics, ERA5 cell, distance to sea, etc.).  
   - Force their `env_class` to **`rural_coastal`** based on visual inspection of the imperviousness map.  
   - Re-print class distributions and station names per category (for transparency in the report).

7. **Quality filtering & ERA5 uniqueness**  
   - Keep only stations with:  
     - RMSE_summer ≤ 3 °C  
     - coverage_total ≥ 60%  
     - good_summers ≥ 2  
   - For each `era5_cell_id`, keep the station with:  
     - lower RMSE_summer, higher coverage_total, higher corr_summer.  
   - Print the **names of stations** per `env_class` after filters.

8. **Initial selection by UHI class**  
   - Target ≈ 3 stations per class (`urban_coastal`, `urban_interior`, `rural_coastal`, `rural_interior`),  
     using RMSE, coverage, and correlation as ranking criteria.  
   - Print the selected station names per class.

9. **Force-add rural coastal reference stations**  
   - Take TARQUINIA, ITRI and MONTALTO_PESCIA from the full dataset.  
   - Ensure they are labelled `rural_coastal`.  
   - Add them to the final selection if they are not already included (even if they failed some filters),  
     to guarantee at least a few **rural_coastal** reference stations justified by visual EO inspection.  
   - Print final counts and station names per class.

---

## **Outputs**

- **`selected_stations_central_italy.csv`**  
- **`selected_stations_central_italy.gpkg`**

These files contain the **final set of reference stations**, including:

- ERA5 cell IDs  
- `urban_fraction`, NDVI, `distance_to_sea_km`  
- UHI environment class (`env_class`)  
- quality metrics (coverage, RMSE, correlation)  

The notebook also logs **which stations belong to each UHI class** at each step  
and explicitly documents the **manual inclusion** of **TARQUINIA, ITRI, MONTALTO_PESCIA**  
as coastal rural reference stations.


In [9]:
# ============================================================
# 00_select_reference_stations_central_italy.ipynb
# Select 10–15 reference stations in Central Italy
# ============================================================

from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

# ----------- 0. PATHS ----------------------------


DATA_DIR     = Path(r"D:\Polimi\GenHack 2025\Data")
STATIONS_DIR = DATA_DIR / "ECA_blend_tx"
ERA5_DIR     = DATA_DIR / "derived-era5-land-daily-statistics"

OUTPUT_DIR   = DATA_DIR / "outputs"
EO_DIR       = DATA_DIR / "EO"
VECTOR_DIR   = DATA_DIR / "Vector"

# Diagnostics + valid stations (from previous script)
DIAG_CSV      = OUTPUT_DIR / "station_diagnostics_italy.csv"
VALID_ST_GPKG = OUTPUT_DIR / "valid_stations_italy.gpkg"

# AOI for Central Italy
AOI_FILE = DATA_DIR / "AOI_central_Italy.shp"

# Imperviousness already computed in QGIS around stations
IMP_STATIONS_FILE = DATA_DIR / "stations_central_italy_3035_impervious.gpkg"

# 🔹 NDVI already computed in QGIS (buffers + zonal stats + NDVI_mean_JJA)
NDVI_STATIONS_FILE = DATA_DIR / "stations_central_italy_ndvi.gpkg"
NDVI_LAYER_NAME    = "ndvi_buffers_1500m"

# Coastline (all Europe) – used to compute distance to sea
COASTLINE_FILE = DATA_DIR / "Europe Coastline" / "EEA_Coastline_20170228.shp"

# Output files of this notebook
SELECTED_CSV  = OUTPUT_DIR / "selected_stations_central_italy.csv"
SELECTED_GPKG = OUTPUT_DIR / "selected_stations_central_italy.gpkg"

# --------------------------------------------------------------------
# 1. PARAMETERS & THRESHOLDS
# --------------------------------------------------------------------

# Quality thresholds
MIN_GOOD_SUMMERS    = 2       # at least 2 good summers
MIN_COVERAGE_TOTAL  = 0.6     # fraction of valid days over 2020–2023
MAX_RMSE_SUMMER     = 3.0     # °C

# 🔸 Urban / rural based on imperviousness quantiles (relative to this AOI)
URBAN_Q = 0.75   # top 25% of urban_fraction → "urban"
RURAL_Q = 0.25   # bottom 25% → "rural"

URBAN_HIGH_THRESHOLD = None   # will be computed from data
URBAN_LOW_THRESHOLD  = None   # will be computed from data

COAST_DISTANCE_KM    = 10.0   # < 10 km = coastal

# Target number of reference stations per UHI class
TARGET_PER_CLASS = {
    "urban_coastal": 3,
    "urban_interior": 3,
    "rural_coastal": 3,
    "rural_interior": 3,
}
# Total target ~12 (within 10–15 requested)

# Stations to force as rural_coastal
FORCED_RURAL_COASTAL_NAMES = ["TARQUINIA", "ITRI", "MONTALTO_PESCIA"]

# --------------------------------------------------------------------
# small helper: print station names by class
# --------------------------------------------------------------------

def print_station_names_by_class(label: str, df: gpd.GeoDataFrame):
    print(f"\n{label} — stations by env_class:")
    if "env_class" not in df.columns:
        print("  (env_class column not found)")
        return
    for env, group in df.groupby("env_class"):
        names = ", ".join(sorted(group["name"].astype(str).unique()))
        print(f"  {env} ({len(group)}): {names}")

# ----------- 2. COASTLINE GLOBAL (for distance to sea) --------------------


try:
    coastline_gdf = gpd.read_file(COASTLINE_FILE)
    coastline_gdf_3035 = coastline_gdf.to_crs("EPSG:3035")
    COASTLINE_UNION_3035 = coastline_gdf_3035.unary_union
except Exception as e:
    print("Warning: could not read coastline file:", e)
    COASTLINE_UNION_3035 = None


# ------------------ 3. HELPER FUNCTION: DISTANCE TO SEA ------------------------------


def compute_distance_to_sea_km(geometry) -> float:
    """
    Compute distance from station to nearest coastline (in km).

    Uses coastline union in EPSG:3035.
    """
    if geometry is None or geometry.is_empty:
        return np.nan
    if COASTLINE_UNION_3035 is None:
        return np.nan

    # Project point to EPSG:3035
    point_gdf = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:4326")
    point_proj = point_gdf.to_crs("EPSG:3035")

    coast_geom = COASTLINE_UNION_3035
    if hasattr(coast_geom, "geom_type") and "Polygon" in coast_geom.geom_type:
        coast_geom = coast_geom.boundary

    dist_m = point_proj.geometry.iloc[0].distance(coast_geom)
    return float(dist_m / 1000.0)

# --------------4. LOAD DIAGNOSTICS, VALID STATIONS, AND AOI; CLIP TO CENTRAL ITALY------------------

# Load station diagnostics
diag_df = pd.read_csv(DIAG_CSV)

# Load valid station geometries
stations_gdf = gpd.read_file(VALID_ST_GPKG)

# Load AOI for Central Italy
aoi_gdf = gpd.read_file(AOI_FILE)

# Ensure CRS match
if stations_gdf.crs != aoi_gdf.crs:
    aoi_gdf = aoi_gdf.to_crs(stations_gdf.crs)

# Merge diagnostics into station geometries on 'station_id'
merged_gdf = stations_gdf.merge(
    diag_df,
    on="station_id",
    how="inner",
    suffixes=("", "_diag")
)

# Filter to valid stations only using the flag from the GPKG
merged_gdf = merged_gdf[merged_gdf["is_valid_station"] == True].copy()

# Clip stations to Central Italy AOI
merged_gdf = gpd.overlay(merged_gdf, aoi_gdf, how="intersection")
merged_gdf = merged_gdf.set_geometry("geometry")

print(f"Number of valid stations in Central Italy AOI (after clip): {len(merged_gdf)}")

# ------------- 5. MERGE IMPERVIOUSNESS-BASED URBAN FRACTION --------------------


imp_gdf = gpd.read_file(IMP_STATIONS_FILE, layer="output2")
print("Columns in impervious GPKG:", imp_gdf.columns.tolist())

# Fix the weird column name "imd_\nmean" → "imd_mean"
if "imd_\nmean" in imp_gdf.columns:
    imp_gdf = imp_gdf.rename(columns={"imd_\nmean": "imd_mean"})
    print("Renamed column 'imd_\\nmean' → 'imd_mean'")

imp_df = imp_gdf[["station_id", "imd_mean"]].copy()

# Merge into merged_gdf (still in EPSG:4326)
merged_gdf = merged_gdf.merge(imp_df, on="station_id", how="left")

# Compute urban_fraction = imd_mean / 100 (0–1)
merged_gdf["urban_fraction"] = merged_gdf["imd_mean"] / 100.0

print("Urban fraction stats (0–1):")
print(merged_gdf["urban_fraction"].describe())

# 🔸 Compute dynamic urban/rural thresholds from imperviousness distribution
URBAN_LOW_THRESHOLD  = merged_gdf["urban_fraction"].quantile(RURAL_Q)
URBAN_HIGH_THRESHOLD = merged_gdf["urban_fraction"].quantile(URBAN_Q)

print(f"Dynamic imperviousness thresholds:")
print(f"  RURAL  (<= {RURAL_Q*100:.0f}th pct) : URBAN_LOW_THRESHOLD  = {URBAN_LOW_THRESHOLD:.4f}")
print(f"  URBAN  (>= {URBAN_Q*100:.0f}th pct) : URBAN_HIGH_THRESHOLD = {URBAN_HIGH_THRESHOLD:.4f}")


# ------------- 6. ASSIGN ERA5 GRID CELL IDs (1 STATION PER ERA5 CELL) --------------

def get_sample_era5_nc(era5_dir: Path) -> Path:
    """Return a sample ERA5 NetCDF file to read the grid from."""
    nc_files = list(era5_dir.glob("**/*.nc"))
    if not nc_files:
        raise FileNotFoundError(f"No NetCDF files found under {era5_dir}")
    return nc_files[0]


def assign_era5_cell_ids(stations: gpd.GeoDataFrame, era5_nc_path: Path) -> gpd.GeoDataFrame:
    ds = xr.open_dataset(era5_nc_path)

    if "latitude" in ds.coords and "longitude" in ds.coords:
        lat_name, lon_name = "latitude", "longitude"
    elif "lat" in ds.coords and "lon" in ds.coords:
        lat_name, lon_name = "lat", "lon"
    else:
        raise ValueError("Could not find latitude/longitude coordinate names in ERA5 dataset.")

    def nearest_cell_id(row):
        lat = row.geometry.y
        lon = row.geometry.x
        point_ds = ds.sel({lat_name: lat, lon_name: lon}, method="nearest")
        cell_lat = float(point_ds[lat_name].values)
        cell_lon = float(point_ds[lon_name].values)
        return f"{cell_lat:.4f}_{cell_lon:.4f}"

    stations = stations.copy()
    stations["era5_cell_id"] = stations.apply(nearest_cell_id, axis=1)

    ds.close()
    return stations


sample_nc = get_sample_era5_nc(ERA5_DIR)
merged_gdf = assign_era5_cell_ids(merged_gdf, sample_nc)

print("Number of unique ERA5 cells in Central Italy AOI:",
      merged_gdf["era5_cell_id"].nunique())


# ------------------------ 7. MERGE NDVI_mean FROM QGIS LAYER + NORMALIZE + DISTANCE TO SEA ---------------------


# 🔹 Load NDVI buffers layer (EPSG:3035, with NDVI_mean_JJA)
ndvi_gdf = gpd.read_file(NDVI_STATIONS_FILE, layer=NDVI_LAYER_NAME)
print("Columns in NDVI GPKG:", ndvi_gdf.columns.tolist())

# Keep only station_id + NDVI_mean_JJA
if "NDVI_mean_JJA" not in ndvi_gdf.columns:
    raise ValueError("NDVI_mean_JJA not found in NDVI layer.")

# Use min–max scaling for NDVI (for descriptive purposes; not used in UHI class)
ndvi_raw = ndvi_gdf[["station_id", "NDVI_mean_JJA"]].copy()
ndvi_raw["NDVI_mean_JJA"] = ndvi_raw["NDVI_mean_JJA"].astype(float)

ndvi_min = ndvi_raw["NDVI_mean_JJA"].min()
ndvi_max = ndvi_raw["NDVI_mean_JJA"].max()

print("Raw NDVI_mean_JJA stats (before normalization):")
print(ndvi_raw["NDVI_mean_JJA"].describe())
print(f"NDVI min = {ndvi_min}, max = {ndvi_max}")

ndvi_raw["NDVI_mean"] = (ndvi_raw["NDVI_mean_JJA"] - ndvi_min) / (ndvi_max - ndvi_min)
ndvi_df = ndvi_raw[["station_id", "NDVI_mean"]].copy()

# Merge NDVI into main stations GeoDataFrame (for EDA only)
merged_gdf = merged_gdf.merge(ndvi_df, on="station_id", how="left")

# Distance to sea (km) using coastline union
merged_gdf["distance_to_sea_km"] = merged_gdf.geometry.apply(compute_distance_to_sea_km)

print("NDVI_mean (0–1, normalized) summary:")
print(merged_gdf["NDVI_mean"].describe())
print("distance_to_sea_km summary:")
print(merged_gdf["distance_to_sea_km"].describe())


# ------ 8. UHI CLASS ASSIGNMENT (imperviousness + distance only) ----------------


def classify_uhi_station_simple(urban_fraction, distance_to_sea_km):
    """
    Classify a station into one of four UHI categories based on:
      - urban_fraction (0–1), using quantile-based thresholds
      - distance_to_sea_km (< COAST_DISTANCE_KM = coastal)

    NDVI is NOT used in the classification (kept only as a descriptive feature).
    """
    if np.isnan(urban_fraction) or np.isnan(distance_to_sea_km):
        return "unknown"

    is_coastal = distance_to_sea_km < COAST_DISTANCE_KM

    if urban_fraction >= URBAN_HIGH_THRESHOLD:
        return "urban_coastal" if is_coastal else "urban_interior"
    elif urban_fraction <= URBAN_LOW_THRESHOLD:
        return "rural_coastal" if is_coastal else "rural_interior"
    else:
        return "unknown"


# First, classify all stations
merged_gdf["env_class"] = merged_gdf.apply(
    lambda row: classify_uhi_station_simple(
        row["urban_fraction"],
        row["distance_to_sea_km"],
    ),
    axis=1,
)

print("UHI environment class distribution (all candidates, BEFORE forcing):")
print(merged_gdf["env_class"].value_counts(dropna=False))
print_station_names_by_class("All stations (before forcing)", merged_gdf)


# --------------- 8b. DEBUG + FORCE TARQUINIA / ITRI / MONTALTO_PESCIA AS RURAL_COASTAL ---------


forced_mask_all = merged_gdf["name"].isin(FORCED_RURAL_COASTAL_NAMES)

print("\n>>> Forced rural-coastal candidates (BEFORE forcing env_class):")
print(
    merged_gdf.loc[forced_mask_all,
                   ["station_id", "name", "urban_fraction",
                    "distance_to_sea_km", "env_class",
                    "good_summers", "coverage_total",
                    "rmse_summer", "era5_cell_id"]]
)

# Force their env_class to rural_coastal (based on visual interpretation)
merged_gdf.loc[forced_mask_all, "env_class"] = "rural_coastal"

print("\n>>> Forced rural-coastal candidates (AFTER forcing env_class):")
print(
    merged_gdf.loc[forced_mask_all,
                   ["station_id", "name", "urban_fraction",
                    "distance_to_sea_km", "env_class",
                    "good_summers", "coverage_total",
                    "rmse_summer", "era5_cell_id"]]
)

print("\nUHI environment class distribution (all candidates, AFTER forcing):")
print(merged_gdf["env_class"].value_counts(dropna=False))
print_station_names_by_class("All stations (after forcing)", merged_gdf)


# ---------------------- 9. QUALITY FILTERING & 1 STATION PER ERA5 CELL -------------


def quality_filter(stations: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    mask = (
        (stations["good_summers"] >= MIN_GOOD_SUMMERS) &
        (stations["coverage_total"] >= MIN_COVERAGE_TOTAL) &
        (stations["rmse_summer"] <= MAX_RMSE_SUMMER)
    )
    return stations[mask].copy()


def pick_best_per_era5_cell(stations: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    stations_sorted = stations.sort_values(
        by=["era5_cell_id", "rmse_summer", "coverage_total", "corr_summer"],
        ascending=[True, True, False, False],
    )
    best_stations = stations_sorted.drop_duplicates(subset="era5_cell_id", keep="first")
    return best_stations


candidates_gdf = quality_filter(merged_gdf)
print(f"\nNumber of candidates after quality filter: {len(candidates_gdf)}")

# See if forced stations survive quality filter
forced_candidates_q = candidates_gdf[candidates_gdf["name"].isin(FORCED_RURAL_COASTAL_NAMES)]
print("\n>>> Forced stations AFTER quality filter:")
print(
    forced_candidates_q[
        ["station_id", "name", "env_class",
         "urban_fraction", "distance_to_sea_km",
         "good_summers", "coverage_total", "rmse_summer"]
    ]
)

candidates_gdf = pick_best_per_era5_cell(candidates_gdf)
print(f"Number of candidates after ERA5-cell uniqueness: {len(candidates_gdf)}")

print("Env_class distribution after quality + ERA5-cell filters:")
print(candidates_gdf["env_class"].value_counts(dropna=False))
print_station_names_by_class("Candidates after filters", candidates_gdf)

# Check if forced stations survived both filters
forced_candidates_final = candidates_gdf[candidates_gdf["name"].isin(FORCED_RURAL_COASTAL_NAMES)]
print("\n>>> Forced stations AFTER quality + ERA5 uniqueness:")
print(
    forced_candidates_final[
        ["station_id", "name", "env_class",
         "urban_fraction", "distance_to_sea_km",
         "good_summers", "coverage_total", "rmse_summer"]
    ]
)


# ------------ 10. SELECT REFERENCE STATIONS BY UHI CLASS ---------------


def select_reference_stations_by_class(stations: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    selected_list = []

    for env_class, target_n in TARGET_PER_CLASS.items():
        class_df = stations[stations["env_class"] == env_class].copy()
        if class_df.empty:
            print(f"No stations available for class {env_class}")
            continue

        class_df = class_df.sort_values(
            by=["rmse_summer", "coverage_total", "corr_summer"],
            ascending=[True, False, False],
        )
        selected_class = class_df.head(target_n)
        selected_list.append(selected_class)

    if not selected_list:
        print("Warning: no stations selected per UHI class. Falling back to global quality selection.")
        fallback = stations.sort_values(
            by=["rmse_summer", "coverage_total", "corr_summer"],
            ascending=[True, False, False],
        ).head(15)
        return fallback

    selected_gdf = gpd.GeoDataFrame(pd.concat(selected_list, ignore_index=True),
                                    crs=stations.crs)

    if len(selected_gdf) > 15:
        selected_gdf = selected_gdf.sort_values(
            by=["rmse_summer", "coverage_total", "corr_summer"],
            ascending=[True, False, False],
        ).head(15)

    return selected_gdf


selected_gdf = select_reference_stations_by_class(candidates_gdf)
print(f"\nNumber of selected reference stations (before forcing extra): {len(selected_gdf)}")
print("UHI class distribution among selected stations (before forcing):")
print(selected_gdf["env_class"].value_counts(dropna=False))
print_station_names_by_class("Selected stations (before forcing)", selected_gdf)

# ----------- 10b. FORCE ADD TARQUINIA / ITRI / MONTALTO_PESCIA TO FINAL SELECTION --------------


# Take their rows from merged_gdf (most complete info)
forced_final = merged_gdf[merged_gdf["name"].isin(FORCED_RURAL_COASTAL_NAMES)].copy()

# Ensure env_class is rural_coastal
forced_final["env_class"] = "rural_coastal"

# Drop those already selected (in case any made it through)
if not selected_gdf.empty:
    forced_final = forced_final[~forced_final["station_id"].isin(selected_gdf["station_id"])]

if forced_final.empty:
    print("\nNo additional forced stations to add (all already in selection).")
else:
    # Align columns
    common_cols = [c for c in selected_gdf.columns if c in forced_final.columns]
    forced_final = forced_final[common_cols]

    selected_gdf = pd.concat([selected_gdf, forced_final], ignore_index=True)

    print("\n>>> Forced rural_coastal stations ADDED to final selection:")
    print(
        forced_final[
            ["station_id", "name", "env_class",
             "urban_fraction", "distance_to_sea_km",
             "good_summers", "coverage_total", "rmse_summer"]
        ]
    )

print(f"\nNumber of selected reference stations (AFTER forcing): {len(selected_gdf)}")
print("UHI class distribution among selected stations (AFTER forcing):")
print(selected_gdf["env_class"].value_counts(dropna=False))
print_station_names_by_class("Selected stations (after forcing)", selected_gdf)


# ---------- 11. EXPORT SELECTED STATIONS -----------------


if len(selected_gdf) > 0:
    selected_gdf = selected_gdf.copy()
    selected_gdf["lat"] = selected_gdf.geometry.y
    selected_gdf["lon"] = selected_gdf.geometry.x

    selected_gdf.drop(columns="geometry").to_csv(SELECTED_CSV, index=False)
    print(f"\nSaved selected stations CSV to: {SELECTED_CSV}")

    selected_gdf.to_file(SELECTED_GPKG, driver="GPKG")
    print(f"Saved selected stations GPKG to: {SELECTED_GPKG}")
else:
    print("No stations selected. Revisit EO features and thresholds.")




C:\Users\mfmr2\AppData\Local\Temp\ipykernel_2876\3661669020.py:97: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  COASTLINE_UNION_3035 = coastline_gdf_3035.unary_union


Number of valid stations in Central Italy AOI (after clip): 124
Columns in impervious GPKG: ['station_id', 'name', 'lon', 'lat', 'coverage_total', 'good_summers', 'bias', 'rmse', 'corr', 'bias_summer', 'rmse_summer', 'corr_summer', 'is_valid_station', 'fid_2', 'station_id_2', 'name_2', 'lon_2', 'lat_2', 'coverage_total_2', 'good_summers_2', 'bias_2', 'rmse_2', 'corr_2', 'bias_summer_2', 'rmse_summer_2', 'corr_summer_2', 'is_valid_station_2', 'imd_\nmean', 'n', 'distance', 'feature_x', 'feature_y', 'nearest_x', 'nearest_y', 'geometry']
Renamed column 'imd_\nmean' → 'imd_mean'
Urban fraction stats (0–1):
count    124.000000
mean       0.046365
std        0.082434
min        0.000000
25%        0.007137
50%        0.016876
75%        0.042274
max        0.601315
Name: urban_fraction, dtype: float64
Dynamic imperviousness thresholds:
  RURAL  (<= 25th pct) : URBAN_LOW_THRESHOLD  = 0.0071
  URBAN  (>= 75th pct) : URBAN_HIGH_THRESHOLD = 0.0423
Number of unique ERA5 cells in Central Italy AOI

## Main outputs

### 1. `selected_stations_central_italy.csv`

A **tabular file (no geometry)** with one row per selected reference station  
(**12 rows** in this final version, 3 per UHI class).

Typical columns:

- Identification & location  
  - `station_id`  
  - `name` (city / station name)  
  - `lat`, `lon` (EPSG:4326)  
  - `era5_cell_id` (ID of the ERA5 grid cell the station represents)

- Spatial / EO features  
  - `NDVI_mean` (0–1, NDVI_mean_JJA, min–max normalized from Sentinel-2)  
  - `urban_fraction` (0–1, mean imperviousness around the station)  
  - `distance_to_sea_km` (km to nearest coastline in EPSG:3035)  
  - `env_class`:
    - `urban_coastal`
    - `urban_interior`
    - `rural_coastal`
    - `rural_interior`

- Diagnostics (from the Italy-wide ECAD–ERA5 script)  
  - `good_summers`, `coverage_total`  
  - `bias`, `rmse`, `corr`  
  - `bias_summer`, `rmse_summer`, `corr_summer`

**This CSV is the control file that the next notebook will read**  
to decide **which stations to keep** when building the daily table.

---

### 2. `selected_stations_central_italy.gpkg`

A **GeoPackage** with the **same rows and attributes** as the CSV, plus geometry:

- Same columns as above  
- `geometry` (point, EPSG:4326)

This file is meant for **QGIS / map visualization**, to inspect:

- where the selected stations are,  
- which **UHI class** each one belongs to,  
- how they are distributed spatially over **Central Italy**.

---

## Final set of selected stations

Balanced design: **3 stations per UHI environment class (12 total)**

- **rural_coastal:**  
  - `ITRI`  
  - `MONTALTO_PESCIA`  
  - `TARQUINIA`

- **rural_interior:**  
  - `CASTEL CELLESI`  
  - `CELLENO`  
  - `VITERBO`

- **urban_coastal:**  
  - `BATTIPAGLIA`  
  - `GROMOLA`  
  - `SABAUDIA`

- **urban_interior:**  
  - `PIGNATORE_MAGGGIORE`  
  - `ROMA_LANCIANI`  
  - `VELLETRI_P._LUNGO`

---

##  How the selection works

The **core selection is fully automatic** and based on:

1. **Diagnostics / data quality**
   - `rmse_summer`, `bias_summer`, `corr_summer`  
   - `coverage_total`, `good_summers`

2. **ERA5-cell uniqueness**
   - Each ERA5 grid cell (`era5_cell_id`) is represented by **at most one station**.  
   - Within each cell, the station with **lower RMSE**, **higher coverage** and **higher correlation** is chosen.

3. **Imperviousness-based UHI classification**
   - `urban_fraction` distribution is used to define:
     - **rural threshold** (bottom 25%)  
     - **urban threshold** (top 25%)
   - Combined with `distance_to_sea_km` (< 10 km = coastal) to classify stations as:
     - `urban_coastal`, `urban_interior`, `rural_coastal`, `rural_interior`, or `unknown`.

Then, **based on visual inspection** of the Copernicus imperviousness map and local context:

- Two additional **rural coastal** stations were **manually included**:
  - `ITRI`, `MONTALTO_PESCIA`
- They have:
  - excellent **coverage** and **number of good summers**,  
  - but **`rmse_summer` > 3 °C**,  
- so they are added **on top of** the automatic selection to better represent  
  **rural coastal environments** along the Tyrrhenian coast.

